In [2]:
# ops_check_func.py
# Usage: import ops_check_func

In [ ]:
# The Get Endpoints example code compacted into a function.
def get_endpoints(url):
    s_response = s.get(url, verify=False)
    s_global_ep = s_response.json()
    tenant_list = []
    epg_list = []
    ep_list = []
    ip_list = []
    mac_list = []
    encap_list = []

    object_list = s_global_ep['imdata']
    for object in object_list:
        #print(object)
        dn = object['fvCEp']['attributes']['dn']
        split_dn = dn.split("/")
        tenant_list.append(split_dn[1])
        epg_list.append(split_dn[-2])
        ep_list.append(split_dn[-1])
        ip_list.append(object['fvCEp']['attributes']['ip'])
        mac_list.append(object['fvCEp']['attributes']['mac'])
        encap_list.append(object['fvCEp']['attributes']['encap'])
    list_of_global_ep = zip(tenant_list, epg_list, ep_list, ip_list, mac_list, encap_list)
    df_input = list(list_of_global_ep)
    df = pd.DataFrame(df_input, columns=("Tenant","EPG or VRF/L3out","Endpoint","IP","MAC Addy","Encap"))
    ep = df[['Tenant','Endpoint']]
    grouped_df = ep.groupby(['Tenant']).count()
    return grouped_df, df

In [ ]:
def get_tenant_endpoints(url):
    epg_response = s.get(epg_url, verify=False)
    epg_json = epg_response.json()
    epg_objects_list = epg_json['imdata']
    tenant_list = []
    ap_list = []
    epg_list = []
    endpoint_list = []
    ip_list = []
    mac_list = []
    encap_list = []
    for epg_object in epg_objects_list:
        dn = epg_object['fvAEPg']['attributes']['dn']
        split_dn = dn.split("/")
        tenant = split_dn[1]
        ap = split_dn[2]
        epg = split_dn[-1]
        children_list = epg_object['fvAEPg']['children']
        endpoint = None
        ip = None
        mac = None
        encap = None
        for child in children_list:
            if "fvCEp" in child:
                endpoint = child['fvCEp']['attributes']['name']
                ip = child['fvCEp']['attributes']['ip']
                mac = child['fvCEp']['attributes']['mac']
                encap = child['fvCEp']['attributes']['encap']
        tenant_list.append(tenant)
        ap_list.append(ap)
        epg_list.append(epg)
        endpoint_list.append(endpoint)
        ip_list.append(ip)
        mac_list.append(mac)
        encap_list.append(encap)
    tn_ep_list = zip(tenant_list, ap_list, epg_list, endpoint_list, ip_list, mac_list, encap_list)
    df_input = list(tn_ep_list)
    df = pd.DataFrame(df_input, columns=("Tenant","AP","EPG","Endpoint","IP","MAC Addy","Encap"))
    df.fillna(value=pd.np.nan, inplace=True)
    sorted_df = df.sort_values(by=['Endpoint'], ascending=False)
    grouped_df = df.groupby(['Tenant']).agg({'EPG':'count','Endpoint':'count'})
    return grouped_df, sorted_df